In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.dummy import DummyClassifier
import time

In [2]:
X_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/X_train_TS25_RS3.csv')
X_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/X_test_TS25_RS3.csv')
y_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/y_train_TS25_RS3.csv')
y_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/y_test_TS25_RS3.csv')

In [3]:
Narrow_X_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_X_train_TS25_RS3.csv')
Narrow_X_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_X_test_TS25_RS3.csv')
Narrow_y_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_y_train_TS25_RS3.csv')
Narrow_y_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_y_test_TS25_RS3.csv')

In [3]:
print('problemtype: medical : Catch all isntances of event, false positives are less of a problem. Maximize: Sensitivity, and Precision')
print('problemtype: spam: Catch many instances of event, but minimize FP')
print('problem type: Artisinal, run time less important than accuracy')
print('problem type: Enterprise, run time is a priority')

problemtype: medical : Catch all isntances of event, false positives are less of a problem. Maximize: Sensitivity, and Precision
problemtype: spam: Catch many instances of event, but minimize FP
problem type: Artisinal, run time less important than accuracy
problem type: Enterprise, run time is a priority


In [4]:
#Models to explore
model_set = ['dummy','SVC', 'SGDClassfier', 'BaggingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'GradientBoostingClassifier', 'HistGradientBoostingClassifier']
model_stats = {}
for each in model_set:
    model_stats[each] ={'confuse' : [], 'training_time' : 0, 'ROC_AUC_Score': 0}
    print(each)

dummy
SVC
SGDClassfier
BaggingClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
HistGradientBoostingClassifier


In [18]:
def timing_buddy(model_to_fit, dictionary_string):
    funky_time_start = time.time()
    model_to_fit.fit(X_train, np.ravel(y_train))
    funky_time_stop = time.time()
    funky_train_time = funky_time_stop - funky_time_start
    model_stats[dictionary_string]['training_time'] = funky_train_time
    

In [32]:
def confused_buddy(model_to_confuse, dictionary_string):
    confuse = confusion_matrix(y_test, model_to_confuse.predict(X_test))
    print(dictionary_string)
    print(confuse)
    model_stats[dictionary_string]['confuse'] = confuse

## Dummy

In [ ]:
def classy_dummy(X_train, y_train, X_test, y_test):
    dummy_clf = DummyClassifier(strategy="most_frequent")
    timing_buddy(dummy_clf, 'dummy')
    

In [33]:
dummy_clf = DummyClassifier(strategy="most_frequent")

In [34]:
timing_buddy(dummy_clf, 'dummy')
print(dummy_clf.score(X_test, y_test))
print(dummy_train_time)

0.8037328238665589
0.047391653060913086


In [35]:
confused_buddy(dummy_clf, 'dummy')

dummy
[[10938     0]
 [ 2671     0]]


In [36]:
model_stats['dummy']['ROC_AUC_Score'] = roc_auc_score(y_test, dummy_clf.predict_proba(X_test)[:, 1])

## SVC

In [37]:
SupportVectorClassifier = SVC()
timing_buddy(SupportVectorClassifier, 'SVC')


In [6]:
SupportVectorClassifier.score(X_test, y_test)

0.8037328238665589

In [38]:
confused_buddy(SupportVectorClassifier, 'SVC')

SVC
[[10938     0]
 [ 2671     0]]


In [41]:
model_stats['SVC']['ROC_AUC_Score'] = roc_auc_score(y_test, SupportVectorClassifier.decision_function(X_test))

## Stochastic Gradiant Descent 

In [ ]:
SGD_clf = SGDClassifier()
timing_buddy(SGD_clf, 'SGDClassfier')

In [11]:
SGD_clf.score(X_test, y_test)

0.8037328238665589

In [13]:
SGD_predict = SGD_clf.predict(X_test)
SGD_Confuse = confusion_matrix(y_test, SGD_predict)
model_stats['SGDClassfier']['confuse'] = SGD_Confuse
print(SGD_Confuse)

[[10938     0]
 [ 2671     0]]


In [14]:
model_stats['SGDClassfier']['ROC_AUC_Score'] = roc_auc_score(y_test, SGD_clf.decision_function(X_test))

## BaggingClassifier

In [39]:
Classy_Baggins_DT = BaggingClassifier(n_estimators=10)
timing_buddy(Classy_Baggins_DT, 'BaggingClassifier')
print(Classy_Baggins_DT.score(X_test, y_test))
Bag_predict = Classy_Baggins_DT.predict(X_test)
confused_buddy(Classy_Baggins_DT, 'BaggingClassifier')

0.7742670291718715
BaggingClassifier
[[10337   601]
 [ 2471   200]]


In [41]:
model_stats['BaggingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, Classy_Baggins_DT.predict_proba(X_test)[:, 1])

## RandomForestClassifier

In [42]:
RandoForest = RandomForestClassifier(max_depth=2)
timing_buddy(RandoForest, 'RandomForestClassifier')
confused_buddy(RandoForest, 'RandomForestClassifier')

RandomForestClassifier
[[10938     0]
 [ 2671     0]]


In [43]:
model_stats['RandomForestClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, RandoForest.predict_proba(X_test)[:, 1])

## AdaBoostClassifier

In [44]:
adaira_the_classifier = AdaBoostClassifier(n_estimators=100)
timing_buddy(adaira_the_classifier, 'AdaBoostClassifier')
confused_buddy(adaira_the_classifier, 'AdaBoostClassifier')

AdaBoostClassifier
[[10933     5]
 [ 2666     5]]


In [45]:
model_stats['AdaBoostClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, adaira_the_classifier.predict_proba(X_test)[:, 1])

## GradientBoostingClassifier

In [50]:
grady_the_boosted = GradientBoostingClassifier(n_estimators=100)
timing_buddy(grady_the_boosted, 'GradientBoostingClassifier')
confused_buddy(grady_the_boosted, 'GradientBoostingClassifier')

GradientBoostingClassifier
[[10932     6]
 [ 2667     4]]


In [51]:
model_stats['GradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, grady_the_boosted.predict_proba(X_test)[:, 1])

## HistGradientBoostingClassifier


In [59]:
#for some reason this model didn't like my functions. So long form it is!
histamine_reaction = HistGradientBoostingClassifier()
    
hist_time_start = time.time()
histamine_reaction.fit(X_train, np.ravel(y_train))
hist_time_stop = time.time()
hist_train_time = hist_time_stop - hist_time_start
model_stats['HistGradientBoostingClassifier']['training_time'] = hist_train_time

confuse = confusion_matrix(y_test, histamine_reaction.predict(X_test))
print('histamine_reaction')
print(confuse)
model_stats['HistGradientBoostingClassifier']['confuse'] = confuse
    


histamine_reaction
[[10937     1]
 [ 2671     0]]


In [60]:
model_stats['HistGradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, histamine_reaction.predict_proba(X_test)[:, 1])

## Model Evaluation

In [64]:
for key, value in model_stats.items():
    print(key)
    for underkey, under_value in value.items():
        print(underkey)
        print(under_value)

dummy
confuse
[[10938     0]
 [ 2671     0]]
training_time
0.017272472381591797
ROC_AUC_Score
0.5
SVC
confuse
[[10938     0]
 [ 2671     0]]
training_time
72.97322130203247
ROC_AUC_Score
0
SGDClassfier
confuse
[[10938     0]
 [ 2671     0]]
training_time
0
ROC_AUC_Score
0.4379346808830056
BaggingClassifier
confuse
[[10337   601]
 [ 2471   200]]
training_time
0.5935509204864502
ROC_AUC_Score
0.5660016509102495
RandomForestClassifier
confuse
[[10938     0]
 [ 2671     0]]
training_time
0.8402276039123535
ROC_AUC_Score
0.620886783743285
AdaBoostClassifier
confuse
[[10933     5]
 [ 2666     5]]
training_time
5.037803888320923
ROC_AUC_Score
0.6238947181209032
GradientBoostingClassifier
confuse
[[10932     6]
 [ 2667     4]]
training_time
1.8453543186187744
ROC_AUC_Score
0.6250329535130755
HistGradientBoostingClassifier
confuse
[[10937     1]
 [ 2671     0]]
training_time
0.32375645637512207
ROC_AUC_Score
0.622746025914143


#precision = tp/(tp+fp)
Maximize this for medical or security scenarios, don't wanna miss actual sicknesses

#recall= tp/(tp+fn) 
maximize this for spam, don't wanna be hiding real emails

specificity(tn/(tn+fp)

Confusion matrix format

[[tp, fn]
[fp, tn]]

In [72]:
def medical_model(dicto):
    bestmodel = ""
    training_time= 0
    precision = 0
    for key, value in model_stats.items():
        for underkey, under_value in value.items():
            tp = dicto[key]['confuse'][0][0]
            fp = dicto[key]['confuse'][1][0]
            temp_precision = tp/(tp + fp)
            if temp_precision > precision:
                precision = temp_precision
                bestmodel = key
                training_time = dicto[key]['training_time']
        return bestmodel, precision, training_time
            
            

In [74]:
med_model, med_precision, med_time = medical_model(model_stats)
print(med_model, med_precision, med_time)

dummy 0.8037328238665589 0.017272472381591797


In [75]:
def spam_model(dicto):
    bestmodel = ""
    training_time= 0
    recall = 0
    for key, value in model_stats.items():
        for underkey, under_value in value.items():
            tp = dicto[key]['confuse'][0][0]
            fn = dicto[key]['confuse'][0][1]
            temp_recall = tp/(tp + fn)
            if temp_recall > recall:
                recall = temp_recall
                bestmodel = key
                training_time = dicto[key]['training_time']
        return bestmodel, recall, training_time

In [76]:
spam_model, spam_precision, spam_time = spam_model(model_stats)
print(spam_model, spam_precision, spam_time)

dummy 1.0 0.017272472381591797
